In [53]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
import re
from pyspark.sql.functions import col, when, last_day, to_date, sum as _sum
from pyspark.sql import functions as F

In [54]:
# Simula los argumentos como si los estuvieras pasando por CLI
sys.argv = [
    'glue_script.py',  # nombre ficticio del script, solo para cumplir con la forma
    '--JOB_NAME', 'ventas_2r',
    '--PAIS', '1',
    '--ANO', '2025',
    '--MES', '1'
]

In [55]:
# Obtener argumentos del job
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'PAIS', 'ANO', 'MES'])

In [56]:
# Inicializar GlueContext y Job
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

logger = glueContext.get_logger()

In [57]:
def select_table_from_rd(table):
    node = glueContext.create_dynamic_frame.from_options(
    connection_type="redshift",
    connection_options={
        "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
        "useConnectionProperties": "true",
        "dbtable": table,
        "connectionName": "conexion-redshift"
    },
    transformation_ctx="node"
    )
    return node

def select_query_from_rd(query):
    node = glueContext.create_dynamic_frame.from_options(
        connection_type="redshift",
        connection_options={
            "sampleQuery": query,
            "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
            "useConnectionProperties": "true",
            "connectionName": "conexion-redshift"
        },
        transformation_ctx="node"
    )
    return node


# QUERYS:

In [58]:
pais = args['PAIS'].zfill(2)
ano = args['ANO']
mes = args['MES'].zfill(2)

query_ventas = f"""
    select
    	to_char(last_day(ut.unidad_fecha), 'yyyymmdd') unidad2,
    	to_char(last_day(ut.unidad_fecha), 'yyyymmdd') periodo, 
    	case
    		when a.agencia_mismas_tiendas = 'INSUMOS' then 'INSUMOS'
    		else tc.tcre_tipo 
    	end tcre_tipo_NUEVO,
    	case 
    		when tcre_tipo_NUEVO in('CREDITO', 'CONTADO') then 1
    		else 0 
    	end es_venta_real,
    	ff.fact_unidad,
    	ut.unidad_fecha,
    	ut.unidad_mes,
    	ut.unidad_ano,
    	a.agencia_codigo,
    	a.agencia_descripcion,
    	a.agencia_pais,
    	c.cliente_codigo,
    	c.hashed_id,
    	c.cliente_descripcion,
    	tc.tcre_codigo,
    	tc.tcre_descripcion,
    	tc.tcre_tipo,
    	ff.fact_costo_merc_vendida,
    	ff.fact_fletes,
    	ff.fact_armado,
    	ff.fact_bodegaje,
    	ff.fact_garantia,
    	ff.fact_valor_venta_oferta
    from
    	gold.gi_fact_facturas ff
    join gold.gi_unidad_tiempo ut
    	on (ff.fact_unidad = ut.unidad_id)
    join gold.gi_agencias a
    	on (ff.fact_agencia = a.agencia_id)
    join gold.gi_clientes c
    	on (ff.fact_cliente = c.CLIENTE_ID)
    join gold.gi_tipo_credito tc
    	on (ff.FACT_TIPO_VENTA = tc.TCRE_ID)
    where
    	a.agencia_pais = '{pais}'
    	and ut.unidad_ano = '{ano}'
    	and ut.unidad_mes = '{mes}'  -- Mes agregado para probar
        
"""

query_contabilidad = f"""
    SELECT
    	f.ryear,
    	CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl01 * 100
	        ELSE f.hsl01
	    END AS hsl01,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl02 * 100
	        ELSE f.hsl02
	    END AS hsl02,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl03 * 100
	        ELSE f.hsl03
	    END AS hsl03,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl04 * 100
	        ELSE f.hsl04
	    END AS hsl04,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl05 * 100
	        ELSE f.hsl05
	    END AS hsl05,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl06 * 100
	        ELSE f.hsl06
	    END AS hsl06,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl07 * 100
	        ELSE f.hsl07
	    END AS hsl07,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl08 * 100
	        ELSE f.hsl08
	    END AS hsl08,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl09 * 100
	        ELSE f.hsl09
	    END AS hsl09,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl10 * 100
	        ELSE f.hsl10
	    END AS hsl10,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl11 * 100
	        ELSE f.hsl11
	    END AS hsl11,
	    CASE 
	        WHEN f.RBUKRS IN ('COMJ', 'COCJ') THEN f.hsl12 * 100
	        ELSE f.hsl12
	    END AS hsl12,
        -- CUENTAS (id = 102)
        cuenta.CUENTA,
        cuenta.campo2  AS rubro,---- COSTOS (Agrupar)
        cuenta.campo3  AS nombre_rubro,
        cuenta.campo5  AS nombre_sub_rubro,
        cuenta.campo6  AS nombre_grupo,
        --Centro de Costo (id = 41)
        ceco.CECO,
        ceco.campo2  AS tipo_ceco, ------ GASTOS (Agrupar)
        ceco.campo3  AS vicepresidencia,
        ceco.campo5  AS UEN, -- unidad estrategica de negocio ()
    	1 as es_venta_real,
        case 
    		when f.RBUKRS IN ('COMJ', 'COCJ') then 01
    		when f.RBUKRS IN ('PAMJ', 'PACJ') then 02
    	end pais
    FROM silver.FAGLFLEXT f
    LEFT JOIN (
        SELECT v.campo1 AS CUENTA, v.*
        FROM bronze_col.variablesrentabilidadexcel v
        JOIN silver.tipos_excel_variables t ON t.id = v.excelid
        WHERE t.nombre = 'CARACTERISTICAS_CUENTAS'
    ) cuenta ON f.RACCT = cuenta.CUENTA
    LEFT JOIN (
        SELECT v.campo1 AS CECO, v.*
        FROM bronze_col.variablesrentabilidadexcel v
        JOIN silver.tipos_excel_variables t ON t.id = v.excelid
        WHERE t.nombre = 'CARACTERISTICAS_CECO'
    ) ceco ON f.RCNTR = ceco.CECO
    WHERE pais = {pais}
      AND f.RYEAR = {ano}
      AND f.RLDNR = '0L'
"""

In [59]:
query_gasto_real = f"""
    with contabilidad as (
SELECT
    	f.ryear,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl01 * 100
    	    ELSE f.hsl01
    	END AS hsl01,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl02 * 100
    	    ELSE f.hsl02
    	END AS hsl02,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl03 * 100
    	    ELSE f.hsl03
    	END AS hsl03,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl04 * 100
    	    ELSE f.hsl04
    	END AS hsl04,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl05 * 100
    	    ELSE f.hsl05
    	END AS hsl05,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl06 * 100
    	    ELSE f.hsl06
    	END AS hsl06,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl07 * 100
    	    ELSE f.hsl07
    	END AS hsl07,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl08 * 100
    	    ELSE f.hsl08
    	END AS hsl08,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl09 * 100
    	    ELSE f.hsl09
    	END AS hsl09,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl10 * 100
    	    ELSE f.hsl10
    	END AS hsl10,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl11 * 100
    	    ELSE f.hsl11
    	END AS hsl11,
    	CASE 
    	    WHEN f.RBUKRS LIKE 'CO%' THEN f.hsl12 * 100
    	    ELSE f.hsl12
    	END AS hsl12,
        -- CUENTAS (id = 102)
        cuenta.CUENTA,
        cuenta.campo2  AS rubro,---- COSTOS (Agrupar)
        cuenta.campo3  AS nombre_rubro,
        cuenta.campo5  AS nombre_sub_rubro,
        cuenta.campo6  AS nombre_grupo,
        --Centro de Costo (id = 41)
        ceco.CECO,
        ceco.campo2  AS tipo_ceco, ------ GASTOS (Agrupar)
        ceco.campo3  AS vicepresidencia,
        ceco.campo5  AS UEN, -- unidad estrategica de negocio ()
        ----------------------------------------------------------------
    	case 
			when rubro in('613500', '613504', '613501', '613502', '613505') then 1
			when rubro in('613503') then 0
		end	es_venta_real, 
    	case 
    		when f.RBUKRS IN ('COMJ', 'COCJ') then 01
    		when f.RBUKRS IN ('PAMJ', 'PACJ') then 02
    	end pais,
    	case 
			when rubro in('613500', '613504') then 'MADERA'
			when rubro in('613503') and pais=1 then 'INSUMOS' -- Solo Colombia tiene INSUMOS
			when rubro in ('613501', '613502', '613505') then 'OTROS COSTOS'
			when nombre_grupo = 'GASTOS' and  tipo_ceco = 'ARMADO' then 'ARMADO'
			when nombre_grupo = 'GASTOS' and  tipo_ceco = 'BODEGAJE' then 'BODEGAJE'
			when rubro in ('414503') and pais=1 then 'BODEGAJE'
			when nombre_grupo = 'GASTOS' and  tipo_ceco = 'EMPAQUE' and pais=1 then 'EMPAQUE'
			when rubro in ('414507') and pais=1 then 'EMPAQUE'
			when nombre_grupo = 'GASTOS' and  tipo_ceco = 'FLETES' then 'FLETES'
			when rubro in ('414502') and pais=1 then 'FLETES'
			when nombre_grupo = 'GASTOS' and  tipo_ceco = 'GARANTIA' then 'GARANTIA'
			when rubro in ('414504') and pais=1 then 'GARANTIA'
		end	tipo_costo 
    FROM silver.FAGLFLEXT f
    LEFT JOIN (
        SELECT v.campo1 AS CUENTA, v.*
        FROM bronze_col.variablesrentabilidadexcel v
        JOIN silver.tipos_excel_variables t ON t.id = v.excelid
        WHERE t.nombre = 'CARACTERISTICAS_CUENTAS'
    ) cuenta ON f.RACCT = cuenta.CUENTA
    LEFT JOIN (
        SELECT v.campo1 AS CECO, v.*
        FROM bronze_col.variablesrentabilidadexcel v
        JOIN silver.tipos_excel_variables t ON t.id = v.excelid
        WHERE t.nombre = 'CARACTERISTICAS_CECO'
    ) ceco ON f.RCNTR = ceco.CECO
    WHERE f.RYEAR = {ano}
      AND f.RLDNR = '0L'
)
select
	to_char(last_day(to_Date('{ano}-{mes}-01', 'yyyy-mm-dd')), 'yyyymmdd') as unidad2,
	tipo_costo,
	es_venta_real,
	case
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 1 then sum(hsl01)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 2 then sum(hsl02)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 3 then sum(hsl03)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 4 then sum(hsl04)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 5 then sum(hsl05)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 6 then sum(hsl06)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 7 then sum(hsl07)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 8 then sum(hsl08)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 9 then sum(hsl09)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 10 then sum(hsl10)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 11 then sum(hsl11)
		when extract(month from to_date(unidad2, 'yyyymmdd')) = 12 then sum(hsl12)
	end as gasto_total
from
	contabilidad
where
	pais = {pais}
    and tipo_costo is not null
group by es_venta_real, tipo_costo
"""

In [60]:
ventas_node = select_query_from_rd(query_ventas) 
ventas_df = ventas_node.toDF()

In [61]:
# Calcula el total del costo de mercancía vendida por mes (unidad2) y tipo de venta (es_venta_real)
costo_merc_vend_df = ventas_df.groupBy("unidad2", "es_venta_real") \
    .agg(_sum("fact_costo_merc_vendida").alias("tot_merc_vend"))

In [63]:
# Calcular proporción de costo mercancía vendida
ventas_df = ventas_df.join(costo_merc_vend_df, on=["unidad2", "es_venta_real"], how="left")

ventas_df = ventas_df.withColumn(
    "prop_cost_merc_vend",
    when(col("tot_merc_vend").isNotNull(),
         col("fact_costo_merc_vendida") / col("tot_merc_vend")).otherwise(0)
)

In [43]:
totales_costos_df = ventas_df.groupBy("unidad2").agg(
    _sum("fact_fletes").alias("tot_fletes"),
    _sum("fact_armado").alias("tot_armado"),
    _sum("fact_bodegaje").alias("tot_bodegaje"),
    _sum("fact_garantia").alias("tot_garantia")
)
totales_costos_df.show(5)

+--------+-------------+------------+-------------+------------+
| unidad2|   tot_fletes|  tot_armado| tot_bodegaje|tot_garantia|
+--------+-------------+------------+-------------+------------+
|20250131|1043370166.00|230925283.00|1572150242.00|384908418.00|
+--------+-------------+------------+-------------+------------+



In [64]:
#Cálculo de proporciones de fletes, armado, bodegaje y garantía
ventas_df = ventas_df.join(totales_costos_df, on="unidad2", how="left")

ventas_df = ventas_df.withColumn(
    "prop_fletes",
    when((col("tot_fletes").isNotNull()) & (col("tot_fletes") != 0),  #se evita dividir entre 0 y NA
         col("fact_fletes") / col("tot_fletes")).otherwise(0)
).withColumn(
    "prop_armado",
    when((col("tot_armado").isNotNull()) & (col("tot_armado") != 0),
         col("fact_armado") / col("tot_armado")).otherwise(0)
).withColumn(
    "prop_bodegaje",
    when((col("tot_bodegaje").isNotNull()) & (col("tot_bodegaje") != 0),
         col("fact_bodegaje") / col("tot_bodegaje")).otherwise(0)
).withColumn(
    "prop_garantia",
    when((col("tot_garantia").isNotNull()) & (col("tot_garantia") != 0),
         col("fact_garantia") / col("tot_garantia")).otherwise(0)
).withColumn(
    "prop_empaque",
    col("prop_armado")
)


### GASTO MERCANCIA VENDIDAD CON CODIGO

In [50]:
def costo_codigo():
    contabilidad_node = select_query_from_rd(query_contabilidad) 
    contabilidad_df = contabilidad_node.toDF()
    
    #Filtrar rubros
    rubros_mercancia_vendida = ["613500", "613504"]
    contabilidad_merc_vend = contabilidad_df.filter(contabilidad_df['rubro'].isin(rubros_mercancia_vendida))
    
    # Especificamos las columnas que queremos sumar
    value_vars = ["hsl01", "hsl02", "hsl03", "hsl04", "hsl05", "hsl06", "hsl07", "hsl08", "hsl09", "hsl10", "hsl11", "hsl12"]
    
    # Agrupamos por 'es_venta_real' y sumamos las columnas
    contabilidad_merc_vend = contabilidad_merc_vend.groupBy("es_venta_real").agg(
        *[ _sum(col).alias(col) for col in value_vars ]
    )
    
    # Renombrar las columnas para mayor claridad
    for col_name in value_vars:
        contabilidad_merc_vend = contabilidad_merc_vend.withColumnRenamed(col_name, f"sum_{col_name}")
    
    for var in value_vars:
        var = f"sum_{var}"
    
    # Sumar todas las columnas de los costos para obtener el gasto total
    contabilidad_merc_vend = contabilidad_merc_vend.withColumn(
        "total_gasto_mv", 
        sum([F.col(f"sum_{col}") for col in value_vars])
    )
    
    fecha_inicio = f"{ano}-{mes}-01"
    
    # Generar la última fecha del mes
    contabilidad_merc_vend = contabilidad_merc_vend.withColumn(
        "unidad2",
        date_format(last_day(to_date(lit(fecha_inicio), 'yyyy-MM-dd')), "yyyyMMdd")
    )
    
    contabilidad_merc_vend = contabilidad_merc_vend.select("es_venta_real", "unidad2", "total_gasto_mv")
    
    #MADERA
    ventas_df_mv_2 = ventas_df.join(contabilidad_merc_vend, on=["unidad2", "es_venta_real"], how="left")
    
    ventas_df_mv_2 = ventas_df_mv.withColumn(
        "madera",
        col("total_gasto_mv") * col("prop_cost_merc_vend")
    )
    
    ventas_df_mv_2.write.option("header", "true").csv("./salida/csv", sep =";")
    print("Archivo generado")

### GASTO MERCANCIA VENDIDA CON QUERY

In [65]:
gasto_real = select_query_from_rd(query_gasto_real)
gasto_real_df = gasto_real.toDF()

## COSTOS DE MERCANCIA VENDIDA

In [66]:
#MADERA
gasto_real_madera_df = gasto_real_df.filter(gasto_real_df["tipo_costo"] == 'MADERA')

gasto_real_madera_df = gasto_real_madera_df.select("unidad2", "es_venta_real", "gasto_total")

gasto_real_madera_df = gasto_real_madera_df.withColumnRenamed("gasto_total", "gasto_total_madera")

ventas_df_gastos = ventas_df.join(gasto_real_madera_df, on=["unidad2", "es_venta_real"], how="left")

ventas_df_gastos = ventas_df_gastos.withColumn(
    "madera",
    col("gasto_total_madera") * col("prop_cost_merc_vend")
)


In [67]:
#INSUMOS
gasto_real_insumos_df = gasto_real_df.filter(gasto_real_df["tipo_costo"] == 'INSUMOS')

gasto_real_insumos_df = gasto_real_insumos_df.select("unidad2", "es_venta_real", "gasto_total")

gasto_real_insumos_df = gasto_real_insumos_df.withColumnRenamed("gasto_total", "gasto_total_insumos")

ventas_df_gastos = ventas_df_gastos.join(gasto_real_insumos_df, on=["unidad2", "es_venta_real"], how="left")

ventas_df_gastos = ventas_df_gastos.withColumn(
    "cost_insumos",
    col("gasto_total_insumos") * col("prop_cost_merc_vend")
)

In [68]:
#OTROS COSTOS
gasto_real_otros_costos_df = gasto_real_df.filter(gasto_real_df["tipo_costo"] == 'OTROS COSTOS')

gasto_real_otros_costos_df = gasto_real_otros_costos_df.select("unidad2", "es_venta_real", "gasto_total")

gasto_real_otros_costos_df = gasto_real_otros_costos_df.withColumnRenamed("gasto_total", "gasto_total_otros_costos")

ventas_df_gastos = ventas_df_gastos.join(gasto_real_otros_costos_df, on=["unidad2", "es_venta_real"], how="left")

ventas_df_gastos = ventas_df_gastos.withColumn(
    "otros_costos",
    col("gasto_total_otros_costos") * col("prop_cost_merc_vend")
)

## OTROS COSTOS VARIABLES

In [69]:
def gastos_otros_costos_variables(df_gastos_total, tipo_costo, nombre_total, llave, join, nombre_col, prop, df_ventas):
    df_gasto = df_gastos_total.filter(gasto_real_df["tipo_costo"] == tipo_costo)

    df_gasto = df_gasto.select("unidad2", "gasto_total")

    df_gasto = df_gasto.withColumnRenamed("gasto_total", nombre_total)

    df_ventas = df_ventas.join(df_gasto, on=llave, how=join)

    df_ventas = df_ventas.withColumn(
        nombre_col,
        col(nombre_total) * col(prop)
    )

    return df_ventas
    

In [70]:
gastos = ["armado",
         "bodegaje",
          "empaque",
          "fletes",
          "garantia"
         ]

for gasto in gastos:
    ventas_df_gastos = gastos_otros_costos_variables(
    df_gastos_total=gasto_real_df, 
    tipo_costo=gasto.upper(), 
    nombre_total=f"gasto_total_{gasto}",
    llave=["unidad2"], 
    join="left", 
    nombre_col=gasto, 
    prop=f"prop_{gasto}", 
    df_ventas=ventas_df_gastos)

ventas_df_gastos = ventas_df_gastos.withColumn(
    "unidad2", 
    to_date(ventas_df_gastos["unidad2"], "yyyyMMdd")
)

ventas_df_gastos = ventas_df_gastos.withColumn(
    "periodo", 
    to_date(ventas_df_gastos["unidad2"], "yyyyMMdd")
)


In [60]:
ventas_df_gastos.write.csv("./salidas/ventas.csv", header=True, sep=";")

In [71]:
# Convert the DataFrame to a DynamicFrame
ventas_dynamic_frame = DynamicFrame.fromDF(ventas_df_gastos, glueContext, "ventas_dynamic_frame")

In [72]:
ventas_node_w = glueContext.write_dynamic_frame.from_options(
    frame=ventas_dynamic_frame,
    connection_type="redshift",
    connection_options={
        "postactions": """
            BEGIN;
            END;
        """,
        "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
        "useConnectionProperties": "true",
        "dbtable": "cf_processing.ventas_2r",
        "connectionName": "conexion-redshift",
        "preactions": f"""
            CREATE TABLE IF NOT EXISTS cf_processing.ventas_2r
            (
                unidad2 DATE,
                es_venta_real INTEGER,
                periodo DATE,
                tcre_tipo_nuevo VARCHAR(255),
                fact_unidad DECIMAL(18, 0),
                unidad_fecha TIMESTAMP,
                unidad_mes DECIMAL(18, 6),
                unidad_ano DECIMAL(18, 6),
                agencia_codigo VARCHAR(255),
                agencia_descripcion VARCHAR(255),
                agencia_pais VARCHAR(255),
                cliente_codigo VARCHAR(255),
                hashed_id VARCHAR(255),
                cliente_descripcion VARCHAR(255),
                tcre_codigo VARCHAR(255),
                tcre_descripcion VARCHAR(255),
                tcre_tipo VARCHAR(255),
                fact_costo_merc_vendida DECIMAL(18, 6),
                fact_fletes DECIMAL(18, 6),
                fact_armado DECIMAL(18, 6),
                fact_bodegaje DECIMAL(18, 6),
                fact_garantia DECIMAL(18, 6),
                tot_merc_vend DECIMAL(18, 6),
                prop_cost_merc_vend DECIMAL(18, 6),
                tot_fletes DECIMAL(18, 6),
                tot_armado DECIMAL(18, 6),
                tot_bodegaje DECIMAL(18, 6),
                tot_garantia DECIMAL(18, 6),
                prop_fletes DECIMAL(18, 6),
                prop_armado DECIMAL(18, 6),
                prop_bodegaje DECIMAL(18, 6),
                prop_garantia DECIMAL(18, 6),
                prop_empaque DECIMAL(18, 6),
                gasto_total_madera DECIMAL(18, 6),
                madera DECIMAL(18, 6),
                gasto_total_insumos DECIMAL(18, 6),
                cost_insumos DECIMAL(18, 6),
                gasto_total_otros_costos DECIMAL(18, 6),
                otros_costos DECIMAL(18, 6),
                gasto_total_armado DECIMAL(18, 6),
                armado DECIMAL(18, 6),
                gasto_total_bodegaje DECIMAL(18, 6),
                bodegaje DECIMAL(18, 6),
                gasto_total_empaque DECIMAL(18, 6),
                empaque DECIMAL(18, 6),
                gasto_total_fletes DECIMAL(18, 6),
                fletes DECIMAL(18, 6),
                gasto_total_garantia DECIMAL(18, 6),
                garantia DECIMAL(18, 6),
                fact_valor_venta_oferta DECIMAL(18, 6)
            );

            DELETE FROM cf_processing.ventas_2r
            where unidad = last_day(to_date('{ano}-{mes}','yyyy-mm'))
            and agencia_pais = '{pais}';
        """
    },
    transformation_ctx="ventas_node_w"
)